In [1]:
import sys
PATH = '/home/epuerta/Seroprevalence'
DIR_DATA = '/home/epuerta/Seroprevalence/'
sys.path.append(PATH) if PATH not in list(sys.path) else None
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import model_selection, preprocessing
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import classification_report
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB 
from mlxtend.classifier import EnsembleVoteClassifier
from sklearn.pipeline import Pipeline
from mlxtend.feature_selection import SequentialFeatureSelector
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv(DIR_DATA + 'MC_CB.csv')

In [3]:
data = data.replace(np.nan, 0)
data.head()

,Edad,Sexo,raza,diagnosticado_covid,estado_civil,actividad_principal,contacto_sosp_covid,comparte_hab_per_cov,cuidador_persona_covid,familiar_fallecido_covid,...,dolor_garganta,fatiga_astenia,perdida_olfato,disminucion_gusto,comportamiento_pandemia,usa_tapabocas,tipo_tapabocas,cuantas_veces_limpia,veces_lava_manos,RESULTADO
0,48.0,2.0,1.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,1.0,4.0,3.0,0.0
1,54.0,1.0,1.0,0.0,1.0,2.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,1.0,2.0,3.0,0.0
2,62.0,2.0,1.0,0.0,4.0,4.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,1.0,1.0,2.0,3.0,0.0
3,22.0,2.0,1.0,0.0,1.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,1.0,2.0,3.0,0.0
4,46.0,1.0,1.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,1.0,2.0,3.0,3.0,0.0


In [4]:
data = data.drop(['raza', 'diagnosticado_covid'], axis = 1)

In [5]:
data.head()

,Edad,Sexo,estado_civil,actividad_principal,contacto_sosp_covid,comparte_hab_per_cov,cuidador_persona_covid,familiar_fallecido_covid,medio_transporte_usa,modalidad_trabajo_pandemia___1,...,dolor_garganta,fatiga_astenia,perdida_olfato,disminucion_gusto,comportamiento_pandemia,usa_tapabocas,tipo_tapabocas,cuantas_veces_limpia,veces_lava_manos,RESULTADO
0,48.0,2.0,4.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,1.0,4.0,3.0,0.0
1,54.0,1.0,1.0,2.0,1.0,0.0,0.0,0.0,7.0,1.0,...,0.0,0.0,0.0,0.0,4.0,1.0,1.0,2.0,3.0,0.0
2,62.0,2.0,4.0,4.0,0.0,0.0,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,5.0,1.0,1.0,2.0,3.0,0.0
3,22.0,2.0,1.0,5.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,4.0,1.0,1.0,2.0,3.0,0.0
4,46.0,1.0,2.0,2.0,0.0,0.0,0.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,5.0,1.0,2.0,3.0,3.0,0.0


In [6]:
array = data.values
array

array([[48.,  2.,  4., ...,  4.,  3.,  0.],
       [54.,  1.,  1., ...,  2.,  3.,  0.],
       [62.,  2.,  4., ...,  2.,  3.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [7]:
encoder = preprocessing.LabelEncoder()
x = preprocessing.normalize(array[:,0:36])
y = encoder.fit_transform(array[:,37])

In [8]:
x

array([[0.9812279 , 0.0408845 , 0.08176899, ..., 0.02044225, 0.02044225,
        0.08176899],
       [0.98639392, 0.01826655, 0.01826655, ..., 0.01826655, 0.01826655,
        0.03653311],
       [0.99013239, 0.03193975, 0.06387951, ..., 0.01596988, 0.01596988,
        0.03193975],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [9]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [10]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(x,y)

In [11]:
seed = 7
num_trees = 100
max_features = 10
n_splits = 10

In [12]:
kfold = StratifiedShuffleSplit(n_splits=n_splits, test_size=0.30, random_state=42)
cl_dt = DecisionTreeClassifier()
cl_rf = RandomForestClassifier()
cl_lr = LogisticRegression()
cl_svm = SVC()
cl_nb = GaussianNB()

In [13]:
estimators = [('Logistic Regression', cl_lr), ('SVM', cl_svm), ('Decision Tree', cl_dt), 
              ('Random Forest', cl_rf), ('Naive Bayes', cl_nb)]
ensamble = {'BaggingClassifier': BaggingClassifier(n_estimators=num_trees, random_state=42),
            'ExtraTrees': ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features),
            'AdaBoost': AdaBoostClassifier(n_estimators=num_trees, random_state=seed),
            'GradientBoosting': GradientBoostingClassifier(n_estimators=num_trees, random_state=seed), 
            'Voting': VotingClassifier(estimators)}

In [14]:
for name, cl in ensamble.items():
    results = model_selection.cross_val_score(cl, x, y, cv=kfold)
    print('{0}: {1}'.format(name, results.mean()))

BaggingClassifier: 0.677533039647577
ExtraTrees: 0.683920704845815
AdaBoost: 0.6698237885462556
GradientBoosting: 0.6900881057268723
Voting: 0.7017621145374451


In [15]:
# Creating a feature-selection-classifier pipeline
seq_feature = SequentialFeatureSelector(cl_lr, k_features=4, forward=True, floating=False, 
                                        scoring='accuracy',verbose=0, cv=kfold)

In [16]:
cl_pipe = Pipeline([('sequential', seq_feature), ('logistic', cl_lr)])
ensemble_vote = EnsembleVoteClassifier(clfs=[cl_pipe, cl_rf, cl_nb], voting='hard')

In [17]:
params = {'pipeline__sequential__k_features': [1, 2, 3],
          'pipeline__logistic__C': [1.0, 100.0],
          'randomforestclassifier__n_estimators': [20, 200]}

grid = GridSearchCV(estimator=ensemble_vote, param_grid=params, cv=kfold)
grid.fit(x, y)

GridSearchCV(cv=StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=0.3,
            train_size=None),
             estimator=EnsembleVoteClassifier(clfs=[Pipeline(steps=[('sequential',
                                                                     SequentialFeatureSelector(cv=StratifiedShuffleSplit(n_splits=10, random_state=42, test_size=0.3,
            train_size=None),
                                                                                               estimator=LogisticRegression(),
                                                                                               k_features=4,
                                                                                               scoring='accuracy')),
                                                                    ('logistic',
                                                                     LogisticRegression())]),
                                                    RandomForestClassifier(),
 

In [18]:
cv_keys = ('mean_test_score', 'std_test_score', 'params')

for r, _ in enumerate(grid.cv_results_['mean_test_score']):
    print("%0.3f +/- %0.2f %r"
          % (grid.cv_results_[cv_keys[0]][r],
             grid.cv_results_[cv_keys[1]][r] / 2.0,
             grid.cv_results_[cv_keys[2]][r]))

0.690 +/- 0.01 {'pipeline__logistic__C': 1.0, 'pipeline__sequential__k_features': 1, 'randomforestclassifier__n_estimators': 20}
0.700 +/- 0.01 {'pipeline__logistic__C': 1.0, 'pipeline__sequential__k_features': 1, 'randomforestclassifier__n_estimators': 200}
0.693 +/- 0.01 {'pipeline__logistic__C': 1.0, 'pipeline__sequential__k_features': 2, 'randomforestclassifier__n_estimators': 20}
0.697 +/- 0.01 {'pipeline__logistic__C': 1.0, 'pipeline__sequential__k_features': 2, 'randomforestclassifier__n_estimators': 200}
0.691 +/- 0.01 {'pipeline__logistic__C': 1.0, 'pipeline__sequential__k_features': 3, 'randomforestclassifier__n_estimators': 20}
0.700 +/- 0.00 {'pipeline__logistic__C': 1.0, 'pipeline__sequential__k_features': 3, 'randomforestclassifier__n_estimators': 200}
0.686 +/- 0.01 {'pipeline__logistic__C': 100.0, 'pipeline__sequential__k_features': 1, 'randomforestclassifier__n_estimators': 20}
0.696 +/- 0.01 {'pipeline__logistic__C': 100.0, 'pipeline__sequential__k_features': 1, 'rand